# Dynamo Software (Data Extraction, Formatting, and Reporting assigment)

In [1]:
# Cell 1: Imports
import re
import os
import pandas as pd
import numpy as np
from datetime import datetime
import psycopg2  # For PostgreSQL
import pymysql   # For MySQL
import sqlalchemy
from sqlalchemy import create_engine
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment
import docx2txt  # For .docx files
import PyPDF2    # For PDF files
import csv
import json
import logging
from dateutil import parser as date_parser  # For flexible date parsing

In [2]:
# Cell 2: Configuration and Logging Setup
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('financial_data_extractor')

# Configuration
CONFIG = {
    "database": {
        "type": "postgresql",  # Change to "mysql" if using MySQL
        "host": "localhost",
        "port": 5432,  # Change to 3306 for MySQL
        "database": "financial_data",
        "user": "postgres",
        "password": "SenkoSQL"
    },
    "extraction": {
        "mandatory_fields": [
            "as_of_date",
            "original_security_name",
            "investment_in_original",
            "investment_in",
            "investment_in_prior",
            "currency"
        ],
        "additional_fields": [
            "sector",
            "risk_rating",
            "maturity_date",
            "yield_percentage",
            "isin",
            "cusip",
            "asset_class",
            "country",
            "region"
        ],
        # Field name variations for pattern matching
        "field_variations": {
            "as_of_date": [
                r"as[\s_-]*of[\s_-]*date",
                r"valuation[\s_-]*date", 
                r"report[\s_-]*date",
                r"date[\s_-]*of[\s_-]*valuation",
                r"statement[\s_-]*date"
            ],
            "original_security_name": [
                r"original[\s_-]*security[\s_-]*name",
                r"security[\s_-]*name", 
                r"instrument[\s_-]*name",
                r"asset[\s_-]*name",
                r"investment[\s_-]*name"
            ],
            "investment_in_original": [
                r"investment[\s_-]*in[\s_-]*\(original\)", 
                r"original[\s_-]*investment[\s_-]*value",
                r"initial[\s_-]*investment",
                r"acquisition[\s_-]*cost",
                r"purchase[\s_-]*value"
            ],
            "investment_in": [
                r"investment[\s_-]*in(?!\s*\()", 
                r"current[\s_-]*investment[\s_-]*value",
                r"market[\s_-]*value",
                r"current[\s_-]*value",
                r"present[\s_-]*value"
            ],
            "investment_in_prior": [
                r"investment[\s_-]*in[\s_-]*\(prior\)", 
                r"prior[\s_-]*investment[\s_-]*value",
                r"previous[\s_-]*value",
                r"value[\s_-]*previous[\s_-]*period",
                r"last[\s_-]*period[\s_-]*value"
            ],
            "currency": [
                r"currency(?!\s*type)", 
                r"currency[\s_-]*type",
                r"currency[\s_-]*code",
                r"denomination",
                r"traded[\s_-]*in"
            ],
            "sector": [
                r"sector",
                r"industry[\s_-]*sector",
                r"business[\s_-]*sector",
                r"market[\s_-]*sector"
            ],
            "risk_rating": [
                r"risk[\s_-]*rating",
                r"risk[\s_-]*level",
                r"risk[\s_-]*assessment",
                r"risk[\s_-]*profile"
            ],
            "maturity_date": [
                r"maturity[\s_-]*date",
                r"expiry[\s_-]*date",
                r"expiration[\s_-]*date",
                r"term[\s_-]*end[\s_-]*date"
            ],
            "yield_percentage": [
                r"yield[\s_-]*percentage",
                r"yield[\s_-]*rate",
                r"yield[\s_-]*\%",
                r"annual[\s_-]*yield",
                r"rate[\s_-]*of[\s_-]*return"
            ],
            "isin": [
                r"isin",
                r"international[\s_-]*securities[\s_-]*identification[\s_-]*number"
            ],
            "cusip": [
                r"cusip",
                r"committee[\s_-]*on[\s_-]*uniform[\s_-]*securities[\s_-]*identification[\s_-]*procedures"
            ],
            "asset_class": [
                r"asset[\s_-]*class",
                r"asset[\s_-]*type",
                r"investment[\s_-]*type",
                r"instrument[\s_-]*class"
            ],
            "country": [
                r"country",
                r"country[\s_-]*of[\s_-]*risk",
                r"country[\s_-]*of[\s_-]*domicile",
                r"domicile"
            ],
            "region": [
                r"region",
                r"geographic[\s_-]*region",
                r"market[\s_-]*region"
            ]
        }
    },
    "output": {
        "excel_file": "extracted_financial_data.xlsx"
    }
}

In [3]:
# Cell 3 (revised): DocumentExtractor Class - Improved Field Extraction
class DocumentExtractor:
    """Handle extraction of data from various document types with robust pattern matching"""
    
    def __init__(self, file_path):
        self.file_path = file_path
        self.file_extension = os.path.splitext(file_path)[1].lower()
        self.raw_text = ""
        self.extracted_data = []
        self.logger = logging.getLogger('financial_data_extractor.DocumentExtractor')
        
    def extract_text(self):
        """Extract raw text from document based on file extension"""
        self.logger.info(f"Extracting text from {self.file_path} (type: {self.file_extension})")
        
        try:
            if self.file_extension == ".docx":
                self.raw_text = docx2txt.process(self.file_path)
            elif self.file_extension == ".pdf":
                with open(self.file_path, "rb") as file:
                    pdf_reader = PyPDF2.PdfReader(file)
                    for page_num in range(len(pdf_reader.pages)):
                        page = pdf_reader.pages[page_num]
                        self.raw_text += page.extract_text()
            elif self.file_extension == ".txt":
                with open(self.file_path, "r", encoding="utf-8", errors="replace") as file:
                    self.raw_text = file.read()
            elif self.file_extension == ".csv":
                with open(self.file_path, 'r', encoding='utf-8', errors="replace") as file:
                    reader = csv.reader(file)
                    for row in reader:
                        self.raw_text += ' '.join(row) + '\n'
            elif self.file_extension == ".json":
                with open(self.file_path, 'r', encoding='utf-8', errors="replace") as file:
                    data = json.load(file)
                    # Convert JSON to text representation
                    self.raw_text = json.dumps(data, indent=2)
            else:
                self.logger.error(f"Unsupported file format: {self.file_extension}")
                raise ValueError(f"Unsupported file format: {self.file_extension}")
            
            self.logger.info(f"Successfully extracted {len(self.raw_text)} characters of text")
            return self.raw_text
            
        except Exception as e:
            self.logger.error(f"Error extracting text from file: {str(e)}")
            raise
    
    def _build_field_pattern(self, field_name):
        """Build a comprehensive regex pattern for a field based on all its variations"""
        variations = CONFIG["extraction"]["field_variations"].get(field_name, [field_name.lower()])
        pattern_parts = []
        
        for variation in variations:
            # Match the field name followed by: colon, equals, dash, or space then colon
            pattern_parts.append(f"(?:{variation})\\s*[:=\\-]\\s*([^\\n:]+)")
        
        # Join all variations with OR (|)
        return '|'.join(pattern_parts)
    
    def _extract_date_global(self):
        """Extract a global as_of_date from the document"""
        date_patterns = [
            # Common date formats in headers or at document beginning
            r"(?:As of|Valuation|Report|Statement)(?:\s+[Dd]ate)?:\s+([0-9]{1,2}[\\/\\-\\.][0-9]{1,2}[\\/\\-\\.][0-9]{2,4})",
            r"(?:As of|Valuation|Report|Statement)(?:\s+[Dd]ate)?:\s+([A-Za-z]+\s+[0-9]{1,2},?\s+[0-9]{2,4})",
            r"[Dd]ate:\s+([0-9]{1,2}[\\/\\-\\.][0-9]{1,2}[\\/\\-\\.][0-9]{2,4})",
            r"[Dd]ate:\s+([A-Za-z]+\s+[0-9]{1,2},?\s+[0-9]{2,4})"
        ]
        
        for pattern in date_patterns:
            date_match = re.search(pattern, self.raw_text, re.IGNORECASE)
            if date_match:
                return date_match.group(1).strip()
        
        return None
    
    def _identify_investment_sections(self):
        """Identify individual investment sections in the document using multiple patterns"""
        # Try different section delimiter patterns
        section_patterns = [
            # Numbered investment sections
            r"(?:#{1,4}|Section|SECTION)\s+(?:INVESTMENT|Investment|Asset|ASSET)\s+(?:\d+|\w+)(?:\s*:)?\s*\n(.*?)(?=(?:#{1,4}|Section|SECTION)\s+(?:INVESTMENT|Investment|Asset|ASSET)\s+(?:\d+|\w+)|$)",
            
            # Sections with clear dividers
            r"(?:[-=*]{3,})\n\s*(?:INVESTMENT|Investment|Asset|ASSET)\s+(?:\d+|\w+)\s*\n(?:[-=*]{3,})\n(.*?)(?=(?:[-=*]{3,})\n\s*(?:INVESTMENT|Investment|Asset|ASSET)\s+(?:\d+|\w+)|$)",
            
            # Sections with empty line dividers
            r"\n\s*(?:INVESTMENT|Investment|Asset|ASSET)\s+(?:\d+|\w+)\s*\n\n(.*?)(?=\n\n\s*(?:INVESTMENT|Investment|Asset|ASSET)\s+(?:\d+|\w+)|$)",
            
            # Named investment sections
            r"(?:Investment|INVESTMENT|Asset|ASSET)(?:\s+in|\s+name)?:\s+([^\n]+)\n(.*?)(?=(?:Investment|INVESTMENT|Asset|ASSET)(?:\s+in|\s+name)?:\s+|$)"
        ]
        
        for pattern in section_patterns:
            sections = re.findall(pattern, self.raw_text, re.DOTALL | re.IGNORECASE)
            if sections:
                # If we found sections using this pattern, return them
                if isinstance(sections[0], tuple):
                    # If pattern captured multiple groups, use the last one
                    return [s[-1] for s in sections]
                return sections
        
        # If no sections found, return the whole document as one section
        return [self.raw_text]
    
    def _extract_table_data(self):
        """Try to extract data from tabular formats in the document"""
        # Look for tabular data patterns - simple CSV-like or fixed width formats
        table_patterns = [
            # Header row followed by data rows
            r"(?:Security\s+Name|Asset\s+Name|Investment)[\s,|]+(?:Original|Initial)[\s,|]+(?:Current|Market)[\s,|]+(?:Prior|Previous)[\s,|]+Currency\s*\n((?:.*\n)+)"
        ]
        
        table_data = []
        
        for pattern in table_patterns:
            matches = re.search(pattern, self.raw_text, re.IGNORECASE)
            if matches:
                table_content = matches.group(1)
                rows = table_content.strip().split('\n')
                
                for row in rows:
                    # Split by common delimiters
                    columns = re.split(r'\s{2,}|,|\||\t', row.strip())
                    if len(columns) >= 5:  # Expecting at least 5 columns for basic data
                        data_dict = {}
                        
                        # Map columns to fields based on position
                        # This is a simple heuristic and might need adjustment
                        if len(columns) >= 1:
                            data_dict["original_security_name"] = columns[0].strip()
                        if len(columns) >= 2:
                            data_dict["investment_in_original"] = columns[1].strip()
                        if len(columns) >= 3:
                            data_dict["investment_in"] = columns[2].strip()
                        if len(columns) >= 4:
                            data_dict["investment_in_prior"] = columns[3].strip()
                        if len(columns) >= 5:
                            data_dict["currency"] = columns[4].strip()
                        
                        table_data.append(data_dict)
        
        return table_data
    
    def _clean_field_value(self, field, value):
        """Clean and standardize field values"""
        if not value:
            return value
            
        value = value.strip()
        
        # Standard field-specific cleaning
        if field in ["investment_in_original", "investment_in", "investment_in_prior"]:
            # Extract numeric part
            matches = re.search(r'[-+]?[0-9,.]+(?:\.[0-9]+)?', value)
            if matches:
                return matches.group(0).replace(',', '')
                
        elif field == "currency":
            # Extract currency code
            matches = re.search(r'\b(USD|EUR|GBP|JPY|CHF|CAD|AUD|NZD|[A-Z]{3})\b', value.upper())
            if matches:
                return matches.group(1)
            return value.upper()
            
        elif field == "yield_percentage":
            # Extract percentage value
            matches = re.search(r'[-+]?[0-9,.]+(?:\.[0-9]+)?', value)
            if matches:
                return matches.group(0).replace(',', '')
                
        return value
    
    def extract_data(self):
        """Extract structured data from the text using multiple strategies"""
        # Make sure we have text to process
        if not self.raw_text:
            self.extract_text()
        
        self.logger.info("Starting data extraction")
        
        # Initialize data dictionaries list
        data_dicts = []
        
        # Global date - looks for the as_of_date field in the whole document
        as_of_date = self._extract_date_global()
        self.logger.info(f"Global as_of_date extracted: {as_of_date}")
        
        # First attempt: Try to extract data from tabular format
        table_data = self._extract_table_data()
        if table_data:
            self.logger.info(f"Extracted {len(table_data)} records from tabular format")
            
            # Add the global date to each record
            for record in table_data:
                if as_of_date:
                    record["as_of_date"] = as_of_date
            
            data_dicts.extend(table_data)
        
        # Second attempt: Find investment sections and extract from each
        if not data_dicts:
            self.logger.info("No tabular data found, trying section-based extraction")
            investment_sections = self._identify_investment_sections()
            self.logger.info(f"Identified {len(investment_sections)} investment sections")
            
            # Build patterns dictionary for all fields
            field_patterns = {}
            for field_name in CONFIG["extraction"]["mandatory_fields"] + CONFIG["extraction"]["additional_fields"]:
                field_patterns[field_name] = self._build_field_pattern(field_name)
            
            # Process each investment section
            for section in investment_sections:
                data_dict = {}
                
                # Add the global date to each investment record
                if as_of_date:
                    data_dict["as_of_date"] = as_of_date
                
                # Extract each field from the section
                for field, pattern in field_patterns.items():
                    match = re.search(pattern, section, re.IGNORECASE)
                    if match:
                        # Use the last group in case there are multiple capture groups
                        data_dict[field] = match.group(match.lastindex or 1).strip()
                
                # Map field variations to standardized field names
                # This is for fields that might be extracted with different names
                field_mapping = {
                    "security_name": "original_security_name",
                    "instrument_name": "original_security_name",
                    "asset_name": "original_security_name",
                    
                    "original_investment_value": "investment_in_original",
                    "initial_investment": "investment_in_original",
                    "acquisition_cost": "investment_in_original",
                    "purchase_value": "investment_in_original",
                    
                    "current_investment_value": "investment_in",
                    "market_value": "investment_in",
                    "current_value": "investment_in",
                    "present_value": "investment_in",
                    
                    "prior_investment_value": "investment_in_prior",
                    "previous_value": "investment_in_prior",
                    "value_previous_period": "investment_in_prior",
                    "last_period_value": "investment_in_prior",
                    
                    "currency_type": "currency",
                    "currency_code": "currency",
                    "denomination": "currency",
                    
                    "risk_level": "risk_rating",
                    "risk_assessment": "risk_rating",
                    "risk_profile": "risk_rating",
                    
                    "expiry_date": "maturity_date",
                    "expiration_date": "maturity_date",
                    "term_end_date": "maturity_date",
                    
                    "yield_rate": "yield_percentage",
                    "yield": "yield_percentage",
                    "annual_yield": "yield_percentage",
                    "rate_of_return": "yield_percentage"
                }
                
                # Standardize field names
                for old_field, new_field in field_mapping.items():
                    if old_field in data_dict and old_field != new_field:
                        if new_field not in data_dict or not data_dict[new_field]:
                            data_dict[new_field] = data_dict[old_field]
                        del data_dict[old_field]
                
                # Clean and standardize field values
                for field, value in list(data_dict.items()):
                    if value:
                        data_dict[field] = self._clean_field_value(field, value)
                
                # Only add if we found at least one field
                if len(data_dict) > 1 or (len(data_dict) == 1 and "as_of_date" not in data_dict):
                    data_dicts.append(data_dict)
        
        # Third attempt: If no investment sections were found, try a more general approach
        if not data_dicts:
            self.logger.info("No investment sections found, trying general extraction")
            
            # Create a record for the as_of_date if found
            if as_of_date:
                data_dicts.append({"as_of_date": as_of_date})
            
            # Build patterns dictionary
            field_patterns = {}
            for field_name in CONFIG["extraction"]["mandatory_fields"] + CONFIG["extraction"]["additional_fields"]:
                field_patterns[field_name] = self._build_field_pattern(field_name)
            
            # Try to find individual fields across the whole document
            for field, pattern in field_patterns.items():
                matches = re.findall(pattern, self.raw_text, re.IGNORECASE)
                for i, match in enumerate(matches):
                    # Create new dictionaries as needed
                    while i >= len(data_dicts):
                        data_dicts.append({})
                    
                    # Handle tuple results from multiple capture groups
                    if isinstance(match, tuple):
                        # Find the first non-empty group
                        for group in match:
                            if group.strip():
                                data_dict[field] = self._clean_field_value(field, group)
                                break
                    else:
                        data_dicts[i][field] = self._clean_field_value(field, match)
        
        # Special handling for CSV or JSON files
        if self.file_extension == ".csv":
            try:
                with open(self.file_path, 'r', encoding='utf-8', errors="replace") as file:
                    reader = csv.DictReader(file)
                    rows = list(reader)
                
                if rows:
                    self.logger.info(f"Extracted {len(rows)} records from CSV file")
                    
                    # Map CSV headers to our field names
                    field_mapping = {
                        "Security Name": "original_security_name",
                        "Asset Name": "original_security_name",
                        "Instrument Name": "original_security_name",
                        "Security": "original_security_name",
                        
                        "Initial Investment": "investment_in_original",
                        "Original Investment": "investment_in_original",
                        "Acquisition Cost": "investment_in_original",
                        "Purchase Value": "investment_in_original",
                        
                        "Market Value": "investment_in",
                        "Current Value": "investment_in",
                        "Present Value": "investment_in",
                        "Current Investment": "investment_in",
                        
                        "Prior Value": "investment_in_prior",
                        "Previous Value": "investment_in_prior",
                        "Last Period Value": "investment_in_prior",
                        "Prior Investment": "investment_in_prior",
                        
                        "Currency Type": "currency",
                        "Currency Code": "currency",
                        "Denomination": "currency",
                        
                        "Risk Level": "risk_rating",
                        "Risk Assessment": "risk_rating",
                        "Risk Profile": "risk_rating",
                        
                        "Expiry Date": "maturity_date",
                        "Expiration Date": "maturity_date",
                        "Term End Date": "maturity_date",
                        
                        "Yield Rate": "yield_percentage",
                        "Yield": "yield_percentage",
                        "Annual Yield": "yield_percentage",
                        "Rate of Return": "yield_percentage",
                        "Yield %": "yield_percentage"
                    }
                    
                    csv_data = []
                    for row in rows:
                        data_dict = {}
                        
                        # Add the global date if found
                        if as_of_date:
                            data_dict["as_of_date"] = as_of_date
                        
                        # Map fields
                        for csv_field, value in row.items():
                            # Try to map the field to our standard field names
                            target_field = field_mapping.get(csv_field, csv_field.lower().replace(' ', '_'))
                            
                            # Clean and standardize the value
                            if value:
                                data_dict[target_field] = self._clean_field_value(target_field, value)
                        
                        if len(data_dict) > 1 or (len(data_dict) == 1 and "as_of_date" not in data_dict):
                            csv_data.append(data_dict)
                    
                    # If we found data in the CSV, use that instead
                    if csv_data:
                        data_dicts = csv_data
            except Exception as e:
                self.logger.warning(f"Error processing CSV file: {str(e)}")
        
        elif self.file_extension == ".json":
            try:
                with open(self.file_path, 'r', encoding='utf-8', errors="replace") as file:
                    json_data = json.load(file)
                
                # Try to extract structured data from JSON
                json_records = []
                
                # Look for common JSON structures
                if isinstance(json_data, dict):
                    # Case 1: {"report_date": "...", "investments": [{...}, {...}]}
                    if "investments" in json_data and isinstance(json_data["investments"], list):
                        report_date = json_data.get("report_date") or json_data.get("as_of_date") or as_of_date
                        
                        for investment in json_data["investments"]:
                            data_dict = {"as_of_date": report_date} if report_date else {}
                            
                            # Map JSON fields to our standard fields
                            field_mapping = {
                                "security_name": "original_security_name",
                                "instrument_name": "original_security_name",
                                "asset_name": "original_security_name",
                                
                                "investment_original": "investment_in_original",
                                "original_investment": "investment_in_original",
                                "investment_original": "investment_in_original",
                                "original_investment": "investment_in_original",
                                "initial_investment": "investment_in_original",
                                "acquisition_cost": "investment_in_original",
                                
                                "investment_current": "investment_in",
                                "current_investment": "investment_in",
                                "market_value": "investment_in",
                                "current_value": "investment_in",
                                
                                "investment_prior": "investment_in_prior",
                                "prior_investment": "investment_in_prior",
                                "previous_value": "investment_in_prior",
                                "last_period_value": "investment_in_prior"
                            }
                            
                            for json_field, value in investment.items():
                                # Map to standard field name if needed
                                field_name = field_mapping.get(json_field, json_field)
                                
                                # Clean and standardize the value
                                if value is not None:
                                    data_dict[field_name] = self._clean_field_value(field_name, str(value))
                            
                            if data_dict:
                                json_records.append(data_dict)
                    
                    # Case 2: {"investments": {"investment1": {...}, "investment2": {...}}}
                    elif "investments" in json_data and isinstance(json_data["investments"], dict):
                        report_date = json_data.get("report_date") or json_data.get("as_of_date") or as_of_date
                        
                        for investment_name, investment_data in json_data["investments"].items():
                            data_dict = {"as_of_date": report_date} if report_date else {}
                            data_dict["original_security_name"] = investment_name
                            
                            for json_field, value in investment_data.items():
                                if value is not None:
                                    data_dict[json_field] = self._clean_field_value(json_field, str(value))
                            
                            if data_dict:
                                json_records.append(data_dict)
                
                # Case 3: Direct list of investments
                elif isinstance(json_data, list):
                    for investment in json_data:
                        if isinstance(investment, dict):
                            data_dict = {}
                            
                            # Add the global date if found
                            if as_of_date:
                                data_dict["as_of_date"] = as_of_date
                            
                            for json_field, value in investment.items():
                                if value is not None:
                                    data_dict[json_field] = self._clean_field_value(json_field, str(value))
                            
                            if data_dict:
                                json_records.append(data_dict)
                
                # If we found records in the JSON, use them
                if json_records:
                    self.logger.info(f"Extracted {len(json_records)} records from JSON file")
                    data_dicts = json_records
            
            except Exception as e:
                self.logger.warning(f"Error processing JSON file: {str(e)}")
        
        # Final validation and cleanup
        validated_data = []
        for record in data_dicts:
            # Only include records with at least some mandatory fields
            mandatory_fields_count = sum(1 for field in CONFIG["extraction"]["mandatory_fields"] if field in record and record[field])
            
            # Keep if we have as_of_date and at least one other mandatory field, or at least 2 mandatory fields
            if (mandatory_fields_count >= 2) or ("as_of_date" in record and mandatory_fields_count >= 1):
                validated_data.append(record)
        
        self.extracted_data = validated_data
        self.logger.info(f"Extracted {len(validated_data)} valid records with data")
        return validated_data

In [4]:
# Cell 4 (revised): DataProcessor Class - Improved Data Formatting
class DataProcessor:
    """Process and format the extracted data with robust type handling and conversions"""
    
    def __init__(self, data_list):
        self.raw_data = data_list
        self.processed_data = []
        self.extraction_stats = {}
        self.logger = logging.getLogger('financial_data_extractor.DataProcessor')
        
    def _format_date(self, date_str):
        """Format a date string to MM/DD/YYYY, handling various input formats"""
        if not date_str or str(date_str).lower() in ["n/a", "na", "none", "null"]:
            return date_str
            
        try:
            # Try parsing with dateutil for flexibility
            date_obj = date_parser.parse(date_str, dayfirst=False, yearfirst=False, fuzzy=True)
            return date_obj.strftime('%m/%d/%Y')
        except Exception as e:
            self.logger.warning(f"Could not parse date '{date_str}': {str(e)}")
            return date_str
    
    def _format_currency(self, value_str):
        """Format a currency value to have 2 decimal places"""
        if not value_str or str(value_str).lower() in ["n/a", "na", "none", "null"]:
            return value_str
            
        try:
            # Remove any non-numeric characters except decimal point and negative sign
            clean_value = re.sub(r'[^\d.-]', '', str(value_str))
            # Format as currency with 2 decimal places
            return "{:.2f}".format(float(clean_value))
        except Exception as e:
            self.logger.warning(f"Could not process currency value '{value_str}': {str(e)}")
            return value_str
    
    def _format_percentage(self, value_str):
        """Format a percentage value to have 2 decimal places"""
        if not value_str or str(value_str).lower() in ["n/a", "na", "none", "null"]:
            return value_str
            
        try:
            # Remove any non-numeric characters except decimal point and negative sign
            clean_value = re.sub(r'[^\d.-]', '', str(value_str))
            # Format as percentage with 2 decimal places
            return "{:.2f}".format(float(clean_value))
        except Exception as e:
            self.logger.warning(f"Could not process percentage value '{value_str}': {str(e)}")
            return value_str
    
    def format_data(self):
        """Format all data according to US standards"""
        self.logger.info("Formatting extracted data")
        
        for item in self.raw_data:
            processed_item = {}
            
            # Process date fields (MM/DD/YYYY)
            for field in [f for f in item.keys() if 'date' in f.lower()]:
                if field in item and item[field]:
                    processed_item[field] = self._format_date(item[field])
            
            # Process currency fields (USD format with 2 decimal places)
            for field in ['investment_in_original', 'investment_in', 'investment_in_prior']:
                if field in item and item[field]:
                    processed_item[field] = self._format_currency(item[field])
            
            # Process yield percentage
            if 'yield_percentage' in item and item['yield_percentage']:
                processed_item['yield_percentage'] = self._format_percentage(item['yield_percentage'])
            
            # Copy other fields as is
            for field in item.keys():
                if field not in processed_item and item[field]:
                    processed_item[field] = item[field]
            
            self.processed_data.append(processed_item)
        
        self.logger.info(f"Formatted {len(self.processed_data)} records")
        return self.processed_data
    
    def calculate_statistics(self):
        """Calculate extraction statistics"""
        self.logger.info("Calculating extraction statistics")
        
        total_records = len(self.processed_data)
        if total_records == 0:
            self.extraction_stats = {
                "total_records": 0,
                "mandatory_fields_extracted": 0,
                "mandatory_fields_percentage": 0,
                "extraction_accuracy": 0,
                "missing_fields": CONFIG["extraction"]["mandatory_fields"],
                "inconsistent_data": [],
                "field_presence": {}
            }
            return self.extraction_stats
        
        # Count mandatory fields
        mandatory_fields = CONFIG["extraction"]["mandatory_fields"]
        mandatory_field_counts = {field: 0 for field in mandatory_fields}
        
        # Track presence of all fields
        all_fields = set()
        field_presence = {}
        
        for record in self.processed_data:
            record_fields = set(record.keys())
            all_fields.update(record_fields)
            
            for field in mandatory_fields:
                if field in record and record[field]:
                    mandatory_field_counts[field] += 1
        
        # Calculate field presence percentages
        for field in all_fields:
            count = sum(1 for record in self.processed_data if field in record and record[field])
            field_presence[field] = {
                "count": count,
                "percentage": (count / total_records) * 100
            }
        
        # Calculate percentages
        total_mandatory_fields = len(mandatory_fields) * total_records
        extracted_mandatory_fields = sum(mandatory_field_counts.values())
        
        mandatory_fields_percentage = (extracted_mandatory_fields / total_mandatory_fields) * 100 if total_mandatory_fields > 0 else 0
        
        # Identify missing and inconsistent data
        missing_fields = []
        for field, count in mandatory_field_counts.items():
            if count < total_records:
                missing_fields.append(f"{field} ({total_records - count} missing)")
        
        # Check for inconsistent data
        inconsistent_data = []
        
        # Example check: Verify if currencies are consistent
        currencies = set()
        for record in self.processed_data:
            if 'currency' in record and record['currency']:
                currencies.add(record['currency'])
        
        if len(currencies) > 1:
            inconsistent_data.append(f"Multiple currencies detected: {', '.join(currencies)}")
        
        # Check date formats for consistency
        date_formats = set()
        for record in self.processed_data:
            if 'as_of_date' in record and record['as_of_date']:
                date_formats.add(self._identify_date_format(record['as_of_date']))
        
        if len(date_formats) > 1:
            inconsistent_data.append(f"Multiple date formats detected: {', '.join(date_formats)}")
        
        # Calculate overall extraction accuracy (weighted by importance)
        # Give more weight to mandatory fields
        mandatory_weight = 0.7
        additional_weight = 0.3
        
        mandatory_accuracy = mandatory_fields_percentage
        
        # Additional fields accuracy (if any are found)
        additional_fields = [f for f in all_fields if f not in mandatory_fields]
        if additional_fields:
            additional_fields_count = sum(field_presence[f]["count"] for f in additional_fields)
            additional_fields_total = len(additional_fields) * total_records
            additional_fields_percentage = (additional_fields_count / additional_fields_total) * 100 if additional_fields_total > 0 else 0
            extraction_accuracy = (mandatory_accuracy * mandatory_weight) + (additional_fields_percentage * additional_weight)
        else:
            extraction_accuracy = mandatory_accuracy
        
        self.extraction_stats = {
            "total_records": total_records,
            "mandatory_fields_extracted": extracted_mandatory_fields,
            "mandatory_fields_percentage": mandatory_fields_percentage,
            "extraction_accuracy": extraction_accuracy,
            "missing_fields": missing_fields,
            "inconsistent_data": inconsistent_data,
            "field_presence": field_presence
        }
        
        self.logger.info(f"Statistics calculated: {mandatory_fields_percentage:.2f}% mandatory fields extracted")
        return self.extraction_stats
    
    def _identify_date_format(self, date_str):
        """Identify the format of a date string"""
        if not date_str or str(date_str).lower() in ["n/a", "na", "none", "null"]:
            return "N/A"
            
        # Check for MM/DD/YYYY
        if re.match(r'\d{1,2}/\d{1,2}/\d{4}', date_str):
            return "MM/DD/YYYY"
        
        # Check for DD/MM/YYYY
        elif re.match(r'\d{1,2}/\d{1,2}/\d{4}', date_str):
            return "DD/MM/YYYY"
        
        # Check for YYYY-MM-DD
        elif re.match(r'\d{4}-\d{1,2}-\d{1,2}', date_str):
            return "YYYY-MM-DD"
        
        # Check for Month DD, YYYY
        elif re.match(r'[A-Za-z]+ \d{1,2},?\s+\d{4}', date_str):
            return "Month DD, YYYY"
        
        return "Unknown format"

In [5]:
# Cell 5 (revised): DataStorage Class - Database Connection Fix
class DataStorage:
    """Store processed data in database and Excel file with error handling"""
    
    def __init__(self, processed_data, stats):
        self.data = processed_data
        self.stats = stats
        self.db_config = CONFIG["database"]
        self.excel_file = CONFIG["output"]["excel_file"]
        self.logger = logging.getLogger('financial_data_extractor.DataStorage')
        
    def create_dataframe(self):
        """Convert processed data to pandas DataFrame"""
        # Normalize the data to handle missing fields
        all_fields = set()
        for record in self.data:
            all_fields.update(record.keys())
        
        normalized_data = []
        for record in self.data:
            normalized_record = {field: record.get(field, None) for field in all_fields}
            normalized_data.append(normalized_record)
        
        return pd.DataFrame(normalized_data)
    
    def store_in_database(self):
        """Store data in SQL database with error handling"""
        self.logger.info(f"Storing data in {self.db_config['type']} database")
        
        try:
            # Create database connection
            if self.db_config["type"] == "postgresql":
                connection_string = f"postgresql://{self.db_config['user']}:{self.db_config['password']}@{self.db_config['host']}:{self.db_config['port']}/{self.db_config['database']}"
            else:  # MySQL
                connection_string = f"mysql+pymysql://{self.db_config['user']}:{self.db_config['password']}@{self.db_config['host']}:{self.db_config['port']}/{self.db_config['database']}"
            
            engine = create_engine(connection_string)
            
            # Drop existing objects to avoid conflicts
            from sqlalchemy import text
            with engine.connect() as connection:
                # Drop the view first, then the table
                connection.execute(text("DROP VIEW IF EXISTS financial_data_stats CASCADE;"))
                connection.execute(text("DROP TABLE IF EXISTS financial_data CASCADE;"))
                connection.commit()
            
            # Convert data to DataFrame
            df = self.create_dataframe()
            
            # Store in database
            df.to_sql('financial_data', engine, if_exists='replace', index=False)
            
            # Create view for statistics - dynamically based on actual columns
            self._create_stats_view(engine, df.columns)
            
            self.logger.info(f"Successfully stored {len(df)} records in database")
            return True
        except Exception as e:
            self.logger.error(f"Database storage error: {str(e)}")
            return False
    
    def _create_stats_view(self, engine, columns):
        """Create a database view with extraction statistics based on actual columns"""
        try:
            # Get the actual column names from the dataframe
            column_cases = []
            
            # Add count for each mandatory field if present in columns
            for field in CONFIG["extraction"]["mandatory_fields"]:
                if field in columns:
                    column_cases.append(f"SUM(CASE WHEN {field} IS NOT NULL THEN 1 ELSE 0 END) AS {field}_count")
            
            # Add other useful stats
            if "currency" in columns:
                column_cases.append("COUNT(DISTINCT currency) AS currency_count_distinct")
            
            # Build the SQL
            view_sql = f"""
            CREATE OR REPLACE VIEW financial_data_stats AS
            SELECT
                COUNT(*) AS total_records,
                {', '.join(column_cases)}
            FROM financial_data;
            """
            
            from sqlalchemy import text
            with engine.connect() as connection:
                connection.execute(text(view_sql))
                connection.commit()
            
            self.logger.info("Successfully created financial_data_stats view")
        except Exception as e:
            self.logger.warning(f"Error creating statistics view: {str(e)}")
    
    def store_in_excel(self):
        """Store data in Excel file with two sheets and formatting"""
        self.logger.info(f"Storing data in Excel file: {self.excel_file}")
        
        try:
            # Convert data to DataFrame - handle empty data gracefully
            df = self.create_dataframe()
            
            # Verify we have data to store
            if len(df) == 0:
                self.logger.warning("No data to store in Excel file")
                # Create a dummy dataframe with mandatory columns to avoid Excel creation errors
                dummy_columns = CONFIG["extraction"]["mandatory_fields"] + CONFIG["extraction"]["additional_fields"]
                df = pd.DataFrame(columns=dummy_columns)
            
            # Create a Pandas Excel writer
            writer = pd.ExcelWriter(self.excel_file, engine='openpyxl')
            
            # Write data to "Extracted Data" sheet
            df.to_excel(writer, sheet_name='Extracted Data', index=False)
            
            # Create statistics DataFrame
            stats_data = {
                "Metric": [
                    "Total Records Processed",
                    "Mandatory Fields Extracted",
                    "Mandatory Fields Percentage",
                    "Extraction Accuracy",
                    "Missing Fields",
                    "Inconsistent Data"
                ],
                "Value": [
                    self.stats["total_records"],
                    self.stats["mandatory_fields_extracted"],
                    f"{self.stats['mandatory_fields_percentage']:.2f}%",
                    f"{self.stats['extraction_accuracy']:.2f}%",
                    ", ".join(self.stats["missing_fields"]) if self.stats["missing_fields"] else "None",
                    ", ".join(self.stats["inconsistent_data"]) if self.stats["inconsistent_data"] else "None"
                ]
            }
            
            # Add field presence statistics
            if "field_presence" in self.stats and self.stats["field_presence"]:
                for field, presence in self.stats["field_presence"].items():
                    stats_data["Metric"].append(f"Field presence: {field}")
                    stats_data["Value"].append(f"{presence['count']} records ({presence['percentage']:.2f}%)")
            
            stats_df = pd.DataFrame(stats_data)
            
            # Write statistics to "Statistics" sheet
            stats_df.to_excel(writer, sheet_name='Statistics', index=False)
            
            # Apply formatting to the Excel file
            self._format_excel_file(writer)
            
            # Save the Excel file
            writer.close()
            
            self.logger.info(f"Successfully stored data in Excel file")
            return True
        except Exception as e:
            self.logger.error(f"Excel storage error: {str(e)}")
            return False
    
    def _format_excel_file(self, writer):
        """Apply formatting to the Excel file for better readability"""
        try:
            workbook = writer.book
            
            # Format Data sheet
            worksheet = workbook['Extracted Data']
            
            # Format headers
            for col in range(1, worksheet.max_column + 1):
                cell = worksheet.cell(row=1, column=col)
                cell.font = Font(bold=True)
                cell.fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")
                cell.alignment = Alignment(horizontal='center')
            
            # Adjust column widths
            for col in range(1, worksheet.max_column + 1):
                max_length = 0
                column = worksheet.column_dimensions[chr(64 + col)]  # A, B, C, etc.
                
                # Find the maximum length in the column
                for row in range(1, worksheet.max_row + 1):
                    cell_value = str(worksheet.cell(row=row, column=col).value or "")
                    if len(cell_value) > max_length:
                        max_length = len(cell_value)
                
                # Set width with some padding
                column.width = max(10, min(50, max_length + 2))
            
            # Format Statistics sheet
            worksheet = workbook['Statistics']
            
            # Format headers
            for col in range(1, worksheet.max_column + 1):
                cell = worksheet.cell(row=1, column=col)
                cell.font = Font(bold=True)
                cell.fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")
                cell.alignment = Alignment(horizontal='center')
            
            # Highlight metrics based on values
            for row in range(2, worksheet.max_row + 1):
                metric_cell = worksheet.cell(row=row, column=1)
                value_cell = worksheet.cell(row=row, column=2)
                
                # Highlight percentage metrics
                if metric_cell.value and "Percentage" in str(metric_cell.value) or "Accuracy" in str(metric_cell.value):
                    value_text = str(value_cell.value or "")
                    if value_text and "%" in value_text:
                        try:
                            percentage = float(value_text.replace("%", ""))
                            if percentage < 50:
                                value_cell.fill = PatternFill(start_color="FFCCCC", end_color="FFCCCC", fill_type="solid")
                            elif percentage < 80:
                                value_cell.fill = PatternFill(start_color="FFFFCC", end_color="FFFFCC", fill_type="solid")
                            else:
                                value_cell.fill = PatternFill(start_color="CCFFCC", end_color="CCFFCC", fill_type="solid")
                        except ValueError:
                            pass
                
                # Highlight missing fields or inconsistent data
                if metric_cell.value and ("Missing Fields" in str(metric_cell.value) or "Inconsistent Data" in str(metric_cell.value)):
                    if value_cell.value and str(value_cell.value) != "None":
                        value_cell.fill = PatternFill(start_color="FFCCCC", end_color="FFCCCC", fill_type="solid")
            
            # Adjust column widths
            for col in range(1, worksheet.max_column + 1):
                max_length = 0
                column = worksheet.column_dimensions[chr(64 + col)]  # A, B, C, etc.
                
                # Find the maximum length in the column
                for row in range(1, worksheet.max_row + 1):
                    cell_value = str(worksheet.cell(row=row, column=col).value or "")
                    if len(cell_value) > max_length:
                        max_length = len(cell_value)
                
                # Set width with some padding
                column.width = max(15, min(75, max_length + 2))
                
        except Exception as e:
            self.logger.warning(f"Error applying Excel formatting: {str(e)}")

In [10]:
# Cell 6 (revised): Create Sample Document with More Variations
def create_sample_document(document_type=".txt"):
    """
    Create a sample financial document for testing with various formats and field variations.
    Returns the path to the created document.
    """
    if document_type == ".txt":
        sample_text = """# QUARTERLY INVESTMENT REPORT
## Confidential Financial Document

Report date: 03/31/2024

### DETAILED INVESTMENTS

#### INVESTMENT 1
Security name: Global Technology Fund Class A
Original investment value: 400,000.00
Current value: 475,250.00
Previous value: 425,800.00
Currency code: USD
Sector: Technology
Risk level: Moderate
Maturity date: N/A
Yield rate: 2.45%

#### INVESTMENT 2
Instrument name: Emerging Markets ETF
Acquisition cost: 200,000.00
Market value: 180,500.75
Value previous period: 194,325.00
Currency: USD
Sector: International
Risk rating: High
Maturity date: N/A
Annual yield: 3.85%

#### INVESTMENT 3
Original security name: US Treasury Bond 2026
Initial investment: 250,000.00
Present value: 250,000.00
Prior investment value: 250,000.00
Currency: USD
Sector: Government
Risk profile: Low
Expiration date: 06/15/2026
Yield %: 4.25
"""
        
        # Create a sample text file
        sample_file_path = "sample_financial_document.txt"
        with open(sample_file_path, 'w') as f:
            f.write(sample_text)
        
    elif document_type == ".csv":
        sample_text = """Security Name,Initial Investment,Market Value,Prior Value,Currency,Sector,Risk Rating,Maturity Date,Yield %
Global Technology Fund Class A,400000.00,475250.00,425800.00,USD,Technology,Moderate,N/A,2.45
Emerging Markets ETF,200000.00,180500.75,194325.00,USD,International,High,N/A,3.85
US Treasury Bond 2026,250000.00,250000.00,250000.00,USD,Government,Low,06/15/2026,4.25
"""
        
        # Create a sample CSV file
        sample_file_path = "sample_financial_document.csv"
        with open(sample_file_path, 'w') as f:
            f.write(sample_text)
            
    elif document_type == ".json":
        sample_data = {
            "report_date": "03/31/2024",
            "investments": [
                {
                    "security_name": "Global Technology Fund Class A",
                    "investment_original": "400000.00",
                    "investment_current": "475250.00",
                    "investment_prior": "425800.00",
                    "currency": "USD",
                    "sector": "Technology",
                    "risk_rating": "Moderate",
                    "maturity_date": "N/A",
                    "yield_percentage": "2.45"
                },
                {
                    "security_name": "Emerging Markets ETF",
                    "investment_original": "200000.00",
                    "investment_current": "180500.75",
                    "investment_prior": "194325.00",
                    "currency": "USD",
                    "sector": "International",
                    "risk_rating": "High",
                    "maturity_date": "N/A",
                    "yield_percentage": "3.85"
                },
                {
                    "security_name": "US Treasury Bond 2026",
                    "investment_original": "250000.00",
                    "investment_current": "250000.00",
                    "investment_prior": "250000.00",
                    "currency": "USD",
                    "sector": "Government",
                    "risk_rating": "Low",
                    "maturity_date": "06/15/2026",
                    "yield_percentage": "4.25"
                }
            ]
        }
        
        # Create a sample JSON file
        sample_file_path = "sample_financial_document.json"
        with open(sample_file_path, 'w') as f:
            json.dump(sample_data, f, indent=2)
    
    elif document_type == ".txt-alt":
        # Alternative text format with different field names and formatting
        sample_text = """INVESTMENT PORTFOLIO SUMMARY
=========================
Statement Date: March 31, 2024

INVESTMENT HOLDINGS
------------------

1. ASSET: Global Technology Fund Class A
   Cost: $400,000.00
   Market Value (Current): $475,250.00
   Market Value (Previous Period): $425,800.00
   Traded in: USD
   Industry Sector: Technology
   Risk Assessment: Moderate
   Term End Date: Not Applicable
   Annual Return: 2.45%

2. ASSET: Emerging Markets ETF
   Cost: $200,000.00
   Market Value (Current): $180,500.75
   Market Value (Previous Period): $194,325.00
   Traded in: USD
   Industry Sector: International
   Risk Assessment: High
   Term End Date: Not Applicable
   Annual Return: 3.85%

3. ASSET: US Treasury Bond 2026
   Cost: $250,000.00
   Market Value (Current): $250,000.00
   Market Value (Previous Period): $250,000.00
   Traded in: USD
   Industry Sector: Government
   Risk Assessment: Low
   Term End Date: 15/06/2026
   Annual Return: 4.25%
"""
        
        # Create an alternative sample text file
        sample_file_path = "sample_financial_document_alt.txt"
        with open(sample_file_path, 'w') as f:
            f.write(sample_text)
    
    else:
        # Create a simple txt file as fallback
        sample_file_path = "sample_financial_document.txt"
        with open(sample_file_path, 'w') as f:
            f.write("As of date: 03/31/2024\nAsset: Sample Asset\nInvestment (original): 100000\nInvestment: 120000\nInvestment (prior): 110000\nCurrency: USD")
    
    return sample_file_path

# Cell 6 (revised): Main Function with Better Error Handling
def main(file_path):
    """Main function to orchestrate the extraction, processing and storage"""
    logger = logging.getLogger('financial_data_extractor.main')
    logger.info(f"Starting extraction process for {file_path}")
    
    try:
        # Step 1: Extract data from document
        logger.info(f"Extracting data from {file_path}...")
        extractor = DocumentExtractor(file_path)
        raw_data = extractor.extract_data()
        
        if not raw_data:
            logger.warning("No data was extracted from the document.")
            # Create empty data and continue to generate statistics
            raw_data = [{}]
        
        logger.info(f"Extracted {len(raw_data)} records.")
        
        # Step 2: Process and format data
        logger.info("Processing and formatting data...")
        processor = DataProcessor(raw_data)
        processed_data = processor.format_data()
        stats = processor.calculate_statistics()
        
        # Step 3: Store data
        logger.info("Storing data...")
        storage = DataStorage(processed_data, stats)
        
        # Attempt to store in database
        db_result = storage.store_in_database()
        if db_result:
            logger.info(f"Data successfully stored in {CONFIG['database']['type']} database.")
        else:
            logger.warning(f"Failed to store data in database. Check your database connection settings.")
        
        # Store in Excel
        excel_result = storage.store_in_excel()
        if excel_result:
            logger.info(f"Data successfully stored in Excel file: {CONFIG['output']['excel_file']}")
        else:
            logger.warning("Failed to store data in Excel file.")
        
        # Step 4: Print statistics
        logger.info("\nExtraction Statistics:")
        logger.info(f"Total Records: {stats['total_records']}")
        logger.info(f"Mandatory Fields Extracted: {stats['mandatory_fields_extracted']}")
        logger.info(f"Mandatory Fields Percentage: {stats['mandatory_fields_percentage']:.2f}%")
        logger.info(f"Extraction Accuracy: {stats['extraction_accuracy']:.2f}%")
        
        if stats['missing_fields']:
            logger.info(f"Missing Fields: {', '.join(stats['missing_fields'])}")
        else:
            logger.info("Missing Fields: None")
        
        if stats['inconsistent_data']:
            logger.info(f"Inconsistent Data: {', '.join(stats['inconsistent_data'])}")
        else:
            logger.info("Inconsistent Data: None")
        
        return True
        
    except Exception as e:
        logger.error(f"Error in extraction process: {str(e)}", exc_info=True)
        return False

In [13]:
# Cell 7: Test Text Document Extraction
# Create and test with a sample text document
sample_path_txt = create_sample_document(".txt")
print(f"Created sample document: {sample_path_txt}")

# Test extraction with the sample document
extractor = DocumentExtractor(sample_path_txt)
extracted_data = extractor.extract_data()

print("\n--- Extracted Raw Data (Text) ---")
for i, item in enumerate(extracted_data):
    print(f"\nRecord {i+1}:")
    for key, value in item.items():
        print(f"  {key}: {value}")

# Process the extracted data
processor = DataProcessor(extracted_data)
processed_data = processor.format_data()
stats = processor.calculate_statistics()

print("\n--- Processed Data (Text) ---")
for i, item in enumerate(processed_data):
    print(f"\nRecord {i+1}:")
    for key, value in item.items():
        print(f"  {key}: {value}")

print("\n--- Extraction Statistics (Text) ---")
print(f"Total Records: {stats['total_records']}")
print(f"Mandatory Fields Extracted: {stats['mandatory_fields_extracted']}")
print(f"Mandatory Fields Percentage: {stats['mandatory_fields_percentage']:.2f}%")
print(f"Extraction Accuracy: {stats['extraction_accuracy']:.2f}%")

if stats['missing_fields']:
    print(f"Missing Fields: {', '.join(stats['missing_fields'])}")
else:
    print("Missing Fields: None")

if stats['inconsistent_data']:
    print(f"Inconsistent Data: {', '.join(stats['inconsistent_data'])}")
else:
    print("Inconsistent Data: None")

2025-05-09 11:56:39,934 - financial_data_extractor.DocumentExtractor - INFO - Extracting text from sample_financial_document.txt (type: .txt)
2025-05-09 11:56:39,936 - financial_data_extractor.DocumentExtractor - INFO - Successfully extracted 856 characters of text
2025-05-09 11:56:39,937 - financial_data_extractor.DocumentExtractor - INFO - Starting data extraction
2025-05-09 11:56:39,938 - financial_data_extractor.DocumentExtractor - INFO - Global as_of_date extracted: 03/31/2024
2025-05-09 11:56:39,938 - financial_data_extractor.DocumentExtractor - INFO - No tabular data found, trying section-based extraction
2025-05-09 11:56:39,939 - financial_data_extractor.DocumentExtractor - INFO - Identified 3 investment sections
2025-05-09 11:56:39,945 - financial_data_extractor.DocumentExtractor - INFO - Extracted 3 valid records with data
2025-05-09 11:56:39,946 - financial_data_extractor.DataProcessor - INFO - Formatting extracted data
2025-05-09 11:56:39,946 - financial_data_extractor.Data

Created sample document: sample_financial_document.txt

--- Extracted Raw Data (Text) ---

Record 1:
  as_of_date: 03/31/2024
  original_security_name: Global Technology Fund Class A
  investment_in_original: 400000.00
  investment_in: 475250.00
  investment_in_prior: 425800.00
  currency: USD
  sector: Technology
  risk_rating: Moderate
  maturity_date: N/A
  yield_percentage: 2.45

Record 2:
  as_of_date: 03/31/2024
  original_security_name: Emerging Markets ETF
  investment_in_original: 200000.00
  investment_in: 180500.75
  investment_in_prior: 194325.00
  currency: USD
  sector: International
  risk_rating: High
  maturity_date: N/A
  yield_percentage: 3.85

Record 3:
  as_of_date: 03/31/2024
  original_security_name: US Treasury Bond 2026
  investment_in_original: 250000.00
  investment_in: 250000.00
  investment_in_prior: 250000.00
  currency: USD
  sector: Government
  risk_rating: Low
  maturity_date: 06/15/2026
  yield_percentage: 4.25

--- Processed Data (Text) ---

Record 1

In [15]:
# Cell 8: Test CSV Document Extraction
# Create and test with a sample CSV document
sample_path_csv = create_sample_document(".csv")
print(f"Created sample document: {sample_path_csv}")

# Test extraction with the sample document
extractor = DocumentExtractor(sample_path_csv)
extracted_data = extractor.extract_data()

print("\n--- Extracted Raw Data (CSV) ---")
for i, item in enumerate(extracted_data):
    print(f"\nRecord {i+1}:")
    for key, value in item.items():
        print(f"  {key}: {value}")

# Process the extracted data
processor = DataProcessor(extracted_data)
processed_data = processor.format_data()
stats = processor.calculate_statistics()

print("\n--- Processed Data (CSV) ---")
for i, item in enumerate(processed_data):
    print(f"\nRecord {i+1}:")
    for key, value in item.items():
        print(f"  {key}: {value}")

print("\n--- Extraction Statistics (CSV) ---")
print(f"Total Records: {stats['total_records']}")
print(f"Mandatory Fields Extracted: {stats['mandatory_fields_extracted']}")
print(f"Mandatory Fields Percentage: {stats['mandatory_fields_percentage']:.2f}%")
print(f"Extraction Accuracy: {stats['extraction_accuracy']:.2f}%")

if stats['missing_fields']:
    print(f"Missing Fields: {', '.join(stats['missing_fields'])}")
else:
    print("Missing Fields: None")

if stats['inconsistent_data']:
    print(f"Inconsistent Data: {', '.join(stats['inconsistent_data'])}")
else:
    print("Inconsistent Data: None")

2025-05-09 11:56:41,177 - financial_data_extractor.DocumentExtractor - INFO - Extracting text from sample_financial_document.csv (type: .csv)
2025-05-09 11:56:41,179 - financial_data_extractor.DocumentExtractor - INFO - Successfully extracted 372 characters of text
2025-05-09 11:56:41,179 - financial_data_extractor.DocumentExtractor - INFO - Starting data extraction
2025-05-09 11:56:41,181 - financial_data_extractor.DocumentExtractor - INFO - Global as_of_date extracted: None
2025-05-09 11:56:41,181 - financial_data_extractor.DocumentExtractor - INFO - No tabular data found, trying section-based extraction
2025-05-09 11:56:41,182 - financial_data_extractor.DocumentExtractor - INFO - Identified 1 investment sections
2025-05-09 11:56:41,183 - financial_data_extractor.DocumentExtractor - INFO - No investment sections found, trying general extraction
2025-05-09 11:56:41,184 - financial_data_extractor.DocumentExtractor - INFO - Extracted 3 records from CSV file
2025-05-09 11:56:41,184 - fin

Created sample document: sample_financial_document.csv

--- Extracted Raw Data (CSV) ---

Record 1:
  original_security_name: Global Technology Fund Class A
  investment_in_original: 400000.00
  investment_in: 475250.00
  investment_in_prior: 425800.00
  currency: USD
  sector: Technology
  risk_rating: Moderate
  maturity_date: N/A
  yield_percentage: 2.45

Record 2:
  original_security_name: Emerging Markets ETF
  investment_in_original: 200000.00
  investment_in: 180500.75
  investment_in_prior: 194325.00
  currency: USD
  sector: International
  risk_rating: High
  maturity_date: N/A
  yield_percentage: 3.85

Record 3:
  original_security_name: US Treasury Bond 2026
  investment_in_original: 250000.00
  investment_in: 250000.00
  investment_in_prior: 250000.00
  currency: USD
  sector: Government
  risk_rating: Low
  maturity_date: 06/15/2026
  yield_percentage: 4.25

--- Processed Data (CSV) ---

Record 1:
  maturity_date: N/A
  investment_in_original: 400000.00
  investment_in: 4

In [17]:
# Cell 9: Test JSON Document Extraction
# Create and test with a sample JSON document
sample_path_json = create_sample_document(".json")
print(f"Created sample document: {sample_path_json}")

# Test extraction with the sample document
extractor = DocumentExtractor(sample_path_json)
extracted_data = extractor.extract_data()

print("\n--- Extracted Raw Data (JSON) ---")
for i, item in enumerate(extracted_data):
    print(f"\nRecord {i+1}:")
    for key, value in item.items():
        print(f"  {key}: {value}")

# Process the extracted data
processor = DataProcessor(extracted_data)
processed_data = processor.format_data()
stats = processor.calculate_statistics()

print("\n--- Processed Data (JSON) ---")
for i, item in enumerate(processed_data):
    print(f"\nRecord {i+1}:")
    for key, value in item.items():
        print(f"  {key}: {value}")

print("\n--- Extraction Statistics (JSON) ---")
print(f"Total Records: {stats['total_records']}")
print(f"Mandatory Fields Extracted: {stats['mandatory_fields_extracted']}")
print(f"Mandatory Fields Percentage: {stats['mandatory_fields_percentage']:.2f}%")
print(f"Extraction Accuracy: {stats['extraction_accuracy']:.2f}%")

if stats['missing_fields']:
    print(f"Missing Fields: {', '.join(stats['missing_fields'])}")
else:
    print("Missing Fields: None")

if stats['inconsistent_data']:
    print(f"Inconsistent Data: {', '.join(stats['inconsistent_data'])}")
else:
    print("Inconsistent Data: None")

2025-05-09 11:56:43,874 - financial_data_extractor.DocumentExtractor - INFO - Extracting text from sample_financial_document.json (type: .json)
2025-05-09 11:56:43,876 - financial_data_extractor.DocumentExtractor - INFO - Successfully extracted 1067 characters of text
2025-05-09 11:56:43,876 - financial_data_extractor.DocumentExtractor - INFO - Starting data extraction
2025-05-09 11:56:43,876 - financial_data_extractor.DocumentExtractor - INFO - Global as_of_date extracted: None
2025-05-09 11:56:43,877 - financial_data_extractor.DocumentExtractor - INFO - No tabular data found, trying section-based extraction
2025-05-09 11:56:43,878 - financial_data_extractor.DocumentExtractor - INFO - Identified 1 investment sections
2025-05-09 11:56:43,880 - financial_data_extractor.DocumentExtractor - INFO - No investment sections found, trying general extraction
2025-05-09 11:56:43,880 - financial_data_extractor.DocumentExtractor - INFO - Extracted 3 records from JSON file
2025-05-09 11:56:43,881 -

Created sample document: sample_financial_document.json

--- Extracted Raw Data (JSON) ---

Record 1:
  as_of_date: 03/31/2024
  original_security_name: Global Technology Fund Class A
  investment_in_original: 400000.00
  investment_in: 475250.00
  investment_in_prior: 425800.00
  currency: USD
  sector: Technology
  risk_rating: Moderate
  maturity_date: N/A
  yield_percentage: 2.45

Record 2:
  as_of_date: 03/31/2024
  original_security_name: Emerging Markets ETF
  investment_in_original: 200000.00
  investment_in: 180500.75
  investment_in_prior: 194325.00
  currency: USD
  sector: International
  risk_rating: High
  maturity_date: N/A
  yield_percentage: 3.85

Record 3:
  as_of_date: 03/31/2024
  original_security_name: US Treasury Bond 2026
  investment_in_original: 250000.00
  investment_in: 250000.00
  investment_in_prior: 250000.00
  currency: USD
  sector: Government
  risk_rating: Low
  maturity_date: 06/15/2026
  yield_percentage: 4.25

--- Processed Data (JSON) ---

Record 

In [19]:
# Cell 10: Full Pipeline Test
# Test the full pipeline with a sample document
sample_path = create_sample_document()  # Default is .txt
print(f"Created sample document: {sample_path}")

# Run the main function
result = main(sample_path)

if result:
    print("\nSuccessfully completed the extraction, processing, and storage pipeline!")
else:
    print("\nPipeline execution encountered issues. Check logs for details.")

2025-05-09 11:56:47,805 - financial_data_extractor.main - INFO - Starting extraction process for sample_financial_document.txt
2025-05-09 11:56:47,805 - financial_data_extractor.main - INFO - Extracting data from sample_financial_document.txt...
2025-05-09 11:56:47,806 - financial_data_extractor.DocumentExtractor - INFO - Extracting text from sample_financial_document.txt (type: .txt)
2025-05-09 11:56:47,808 - financial_data_extractor.DocumentExtractor - INFO - Successfully extracted 856 characters of text
2025-05-09 11:56:47,808 - financial_data_extractor.DocumentExtractor - INFO - Starting data extraction
2025-05-09 11:56:47,808 - financial_data_extractor.DocumentExtractor - INFO - Global as_of_date extracted: 03/31/2024
2025-05-09 11:56:47,809 - financial_data_extractor.DocumentExtractor - INFO - No tabular data found, trying section-based extraction
2025-05-09 11:56:47,810 - financial_data_extractor.DocumentExtractor - INFO - Identified 3 investment sections
2025-05-09 11:56:47,810

Created sample document: sample_financial_document.txt

Successfully completed the extraction, processing, and storage pipeline!


In [21]:
# Cell 11: Store in Excel and Database
# Test the storage functionality with the sample data
storage = DataStorage(processed_data, stats)

# Store in Excel
excel_result = storage.store_in_excel()
if excel_result:
    print(f"\nData successfully stored in Excel file: {CONFIG['output']['excel_file']}")
    # Read and display Excel file contents to verify
    try:
        df = pd.read_excel(CONFIG['output']['excel_file'], sheet_name='Extracted Data')
        print("\n--- Excel Data Preview ---")
        display(df)  # Jupyter will nicely format this
        
        stats_df = pd.read_excel(CONFIG['output']['excel_file'], sheet_name='Statistics')
        print("\n--- Excel Statistics Preview ---")
        display(stats_df)  # Jupyter will nicely format this
    except Exception as e:
        print(f"Error reading Excel file: {e}")
else:
    print("Failed to store data in Excel file.")

# Store in database
try:
    # Ensure view is dropped to avoid conflicts
    connection_string = f"postgresql://{CONFIG['database']['user']}:{CONFIG['database']['password']}@{CONFIG['database']['host']}:{CONFIG['database']['port']}/{CONFIG['database']['database']}"
    engine = create_engine(connection_string)
    
    from sqlalchemy import text
    with engine.connect() as connection:
        connection.execute(text("DROP VIEW IF EXISTS financial_data_stats CASCADE;"))
        connection.commit()
    
    print("Successfully dropped the view (if it existed).")
except Exception as e:
    print(f"Error dropping view: {e}")

# Store data in database and verify
try:
    # Step 1: Store the data
    db_result = storage.store_in_database()
    if db_result:
        print(f"Data successfully stored in {CONFIG['database']['type']} database.")
        
        connection_string = f"postgresql://{CONFIG['database']['user']}:{CONFIG['database']['password']}@{CONFIG['database']['host']}:{CONFIG['database']['port']}/{CONFIG['database']['database']}"
        engine = create_engine(connection_string)
        
        # Step 2: Verify the data
        query = "SELECT * FROM financial_data"
        db_data = pd.read_sql(query, engine)
        
        print("\n--- Database Data Preview ---")
        display(db_data)
        
        # Step 3: Query the stats view
        query = "SELECT * FROM financial_data_stats"
        stats_data = pd.read_sql(query, engine)
        
        print("\n--- Database Stats View Preview ---")
        display(stats_data)
    else:
        print(f"Failed to store data in database.")
except Exception as e:
    print(f"Database operation error: {e}")

2025-05-09 11:56:52,064 - financial_data_extractor.DataStorage - INFO - Storing data in Excel file: extracted_financial_data.xlsx
2025-05-09 11:56:52,076 - financial_data_extractor.DataStorage - INFO - Successfully stored data in Excel file



Data successfully stored in Excel file: extracted_financial_data.xlsx

--- Excel Data Preview ---


,currency,risk_rating,as_of_date,investment_in_prior,yield_percentage,maturity_date,investment_in_original,original_security_name,sector,investment_in
0,USD,Moderate,03/31/2024,425800.0,2.45,NaN,400000.0,Global Technology Fund Class A,Technology,475250.00
1,USD,High,03/31/2024,194325.0,3.85,NaN,200000.0,Emerging Markets ETF,International,180500.75
2,USD,Low,03/31/2024,250000.0,4.25,06/15/2026,250000.0,US Treasury Bond 2026,Government,250000.00



--- Excel Statistics Preview ---


,Metric,Value
0,Total Records Processed,3
1,Mandatory Fields Extracted,18
2,Mandatory Fields Percentage,100.00%
3,Extraction Accuracy,100.00%
4,Missing Fields,NaN
5,Inconsistent Data,NaN
6,Field presence: as_of_date,3 records (100.00%)
7,Field presence: investment_in_prior,3 records (100.00%)
8,Field presence: yield_percentage,3 records (100.00%)
9,Field presence: maturity_date,3 records (100.00%)


2025-05-09 11:56:52,137 - financial_data_extractor.DataStorage - INFO - Storing data in postgresql database
2025-05-09 11:56:52,185 - financial_data_extractor.DataStorage - INFO - Successfully created financial_data_stats view
2025-05-09 11:56:52,186 - financial_data_extractor.DataStorage - INFO - Successfully stored 3 records in database


Successfully dropped the view (if it existed).
Data successfully stored in postgresql database.

--- Database Data Preview ---


,currency,risk_rating,as_of_date,investment_in_prior,yield_percentage,maturity_date,investment_in_original,original_security_name,sector,investment_in
0,USD,Moderate,03/31/2024,425800.00,2.45,N/A,400000.00,Global Technology Fund Class A,Technology,475250.00
1,USD,High,03/31/2024,194325.00,3.85,N/A,200000.00,Emerging Markets ETF,International,180500.75
2,USD,Low,03/31/2024,250000.00,4.25,06/15/2026,250000.00,US Treasury Bond 2026,Government,250000.00



--- Database Stats View Preview ---


,total_records,as_of_date_count,original_security_name_count,investment_in_original_count,investment_in_count,investment_in_prior_count,currency_count,currency_count_distinct
0,3,3,3,3,3,3,3,1
